In [ ]:
%run D:\valkyrie_shared\python\lib\valkyrie\run_notebook_import.py

In [ ]:
instrs = stocks_good_dvd()
#instrs = ['AGNCM']
feature_mgr = FeatureMgr('20210101', '20210630', instrs + ['PFF'], 'sampled_for_ml_ii')
sampled_freq_s = 300
n_sample_per_day = TRADING_SEC_PER_DAY / sampled_freq_s
ret_ns = [round(i * n_sample_per_day) for i in [1] ]

ret_col, s2_col = 'risk_tv', 'cywa_mkt_s2h'
df_tv = feature_mgr.calc_rets_('log', ret_col, ret_ns, s2_col, wmul = 1e-7)
df_tv['parent'] = df_tv['ticker'].apply(parent)

ret_col, s2_col = 'cywa_mkt_xh', 'cywa_mkt_s2h'
df_cy = feature_mgr.calc_rets_('log', ret_col, ret_ns, s2_col, wmul = 1e-7)
df_cy['parent'] = df_cy['ticker'].apply(parent)
gc.collect()

In [ ]:
ret_ns

In [ ]:
ins_nbs2 = []
for stk in instrs:
    ins_nbs2.append(ml_data.InsNB(stk,   ['PFF']))

# Test Net

In [ ]:
xcols = ['cywa_mkt_xh']
ycol = 'risk_tv_ret_78_n'
wcol = 'wgt_cywa_mkt_xh_ret_78_n'

In [ ]:
T = 1
mrf = ml_data.MultiRecField(feature_mgr.stk2df, ins_nbs2, xcols, ycol, wcol, T)
dm = ml_data.DataModule.from_dataset(mrf, None, 1024)

In [ ]:
class AvgNetII(ml_modules.Regressor):
  def __init__(self, lr):
    print("LeNet Init")
    super().__init__('l1')
    self.save_hyperparameters()
    self.net = nn.Sequential(
      nn.Flatten(),
      nn.Linear(2,1, bias=False)
      )

In [ ]:
net = AvgNetII(lr = 1e-2)
X = mrf[0][0]
ml_utils.layer_summary(net, X.shape)
ml_utils.parameter_summary(net)

In [ ]:
len(mrf)

In [ ]:
net.apply_init([next(iter(dm.get_dataloader(True)))[0]], ml_utils.init_cnn)

In [ ]:
trainer = ml_modules.Trainer(max_epochs=80, num_gpus=1)
with Timer('training'):
    trainer.fit(net, dm)

In [ ]:
r2, abs_loss, abs_zr_loss, X, Xnet, Y, W, lr = mrf_linear_fit(mrf)

In [ ]:
trainer.loss_total

In [ ]:
def calc_net_loss(net, X,Y,W):
    X = torch.tensor(X, dtype=torch.float32).unsqueeze(-1)

    YW[:,0] = net(X.to(ml_utils.gpu())).view(-1)

    YW = torch.empty(Y.shape[0],2, dtype=torch.float32).to(ml_utils.gpu())

    YW[:,1] = torch.tensor(W)


In [ ]:
X.shape

In [ ]:
trainer.loss_total

In [ ]:
r2, abs_loss, abs_zr_loss, X, Y, W, lr = mrf_linear_fit(mrf)

In [ ]:
mrf[0][0].shape

In [ ]:
Xx = torch.tensor(X.reshape(-1,2,1), dtype=torch.float32)

In [ ]:
Xx.to(ml_utils.gpu())

In [ ]:
YW = torch.empty(Y.shape[0],2, dtype=torch.float32).to(ml_utils.gpu())
YW[:,0] = net(Xx.to(ml_utils.gpu())).view(-1)
YW[:,1] = torch.tensor(W)

In [ ]:
net(Xx.to(ml_utils.gpu())).shape

In [ ]:
def eval_init_loss(net, dm):
    init_loss, loss = 0.0, 0.0
    dl = dm.get_dataloader(True)
    for i, batch in enumerate(dl):
        X, YW = batch[0], batch[-1]                
        ZEROS = torch.zeros(YW.shape[0])
        with torch.no_grad():
            init_loss += net.loss(ZEROS, YW)    
            Y_hat = net(X.to(ml_utils.gpu()))
            loss += net.loss(Y_hat, YW.to(ml_utils.gpu()))
            #Y_hat = lr.predict(X.view(-1, 6)) #net(X.to(ml_utils.gpu())).to(ml_utils.cpu())
            #Y_hat = torch.tensor(Y_hat)
            #Y_hat = YW[:,0]
            #loss += net.loss(Y_hat, YW)            
            
    return init_loss, loss

In [ ]:
[k for k in net.net[1].named_parameters()]

In [ ]:
eval_init_loss(net, dm)

In [ ]:
Y_hat = lr.predict(X)

In [ ]:
YW = torch.empty(Y.shape[0],2)
YW[:,0] = torch.tensor(Y)
YW[:,1] = torch.tensor(W)
Z = torch.zeros(Y.shape[0])

y_hat = Y_hat
Y_hat = torch.tensor(y_hat)

In [ ]:
Y_hatN = net(torch.tensor(X, dtype=torch.float32).to(ml_utils.gpu()))

In [ ]:
net.loss(Y_hatN, YW.to(ml_utils.gpu()))

In [ ]:
Y

In [ ]:
Y_hatN

In [ ]:
W

In [ ]:
torch.sum(torch.abs(Y_hatN.view(-1)-torch.tensor(Y).to(ml_utils.gpu())) * torch.tensor(W).to(ml_utils.gpu()))

In [ ]:
y_hatn = Y_hatN.detach().cpu().numpy().reshape(-1)

In [ ]:
dfxx = pd.DataFrame({'y_hatn' : y_hatn, 'y_hat':y_hat, 'w':W})

In [ ]:
dfxx.describe()

In [ ]:
net.loss(Y_hatN, YW)

In [ ]:
Y_hat.shape

In [ ]:
Y_hatN.shape